In [17]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
import nltk

nltk.download('punkt')



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ilinc\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
categoryIndex = {
    'agriculture' :1,
    'cultural' : 2,
    'parenting' : 3,
    'sport' : 4,
    'travel' : 5
}

ro_df = pd.DataFrame(columns=['category','siteIdx', 'text'])

ro_df_agriculture = pd.read_csv('RO/agricultureRo.csv')
ro_df_agriculture['category'] = categoryIndex['agriculture']
ro_df = pd.concat([ro_df[['category', 'siteIdx', 'text']], ro_df_agriculture[['category', 'siteIdx', 'text']]], ignore_index=True)

ro_df_cultural = pd.read_csv('RO/culturalRo.csv')
ro_df_cultural['category'] = categoryIndex['cultural']
ro_df = pd.concat([ro_df[['category', 'siteIdx', 'text']], ro_df_cultural[['category', 'siteIdx', 'text']]], ignore_index=True)

ro_df_parenting = pd.read_csv('RO/parentingRo.csv')
ro_df_parenting['category'] = categoryIndex['parenting']
ro_df = pd.concat([ro_df[['category', 'siteIdx', 'text']], ro_df_parenting[['category', 'siteIdx', 'text']]], ignore_index=True)

ro_df_sport = pd.read_csv('RO/sportRo.csv')
ro_df_sport['category'] = categoryIndex['sport']
ro_df = pd.concat([ro_df[['category', 'siteIdx', 'text']], ro_df_sport[['category', 'siteIdx', 'text']]], ignore_index=True)

ro_df_travel = pd.read_csv('RO/travelRo.csv')
ro_df_travel['category'] = categoryIndex['travel']
ro_df = pd.concat([ro_df[['category', 'siteIdx', 'text']], ro_df_travel[['category', 'siteIdx', 'text']]], ignore_index=True)

ro_df.to_csv('RomanianTexts.csv', index=False)

md_df = pd.DataFrame(columns=['category','siteIdx', 'text'])

md_df_agriculture = pd.read_csv('MD/agricultureMd.csv')
md_df_agriculture['category'] = categoryIndex['agriculture']
md_df = pd.concat([md_df[['category', 'siteIdx', 'text']], md_df_agriculture[['category', 'siteIdx', 'text']]], ignore_index=True)

md_df_cultural = pd.read_csv('MD/culturalMd.csv')
md_df_cultural['category'] = categoryIndex['cultural']
md_df = pd.concat([md_df[['category', 'siteIdx', 'text']], md_df_cultural[['category', 'siteIdx', 'text']]], ignore_index=True)

md_df_parenting = pd.read_csv('MD/parentingMd.csv')
md_df_parenting['category'] = categoryIndex['parenting']
md_df = pd.concat([md_df[['category', 'siteIdx', 'text']], md_df_parenting[['category', 'siteIdx', 'text']]], ignore_index=True)

md_df_sport = pd.read_csv('MD/sportMd.csv')
md_df_sport['category'] = categoryIndex['sport']
md_df = pd.concat([md_df[['category', 'siteIdx', 'text']], md_df_sport[['category', 'siteIdx', 'text']]], ignore_index=True)

md_df_travel = pd.read_csv('MD/travelMd.csv')
md_df_travel['category'] = categoryIndex['travel']
md_df = pd.concat([md_df[['category', 'siteIdx', 'text']], md_df_travel[['category', 'siteIdx', 'text']]], ignore_index=True)
md_df.to_csv('MoldovanTexts.csv', index=False)

In [47]:
ro_df = pd.read_csv('RomanianTexts.csv')
md_df = pd.read_csv('MoldovanTexts.csv')

file_path = 'stopwords.txt'

with open(file_path, 'r', encoding='utf-8') as file:
    stopwords = file.readlines()

stopwords = [word.strip() for word in stopwords]

In [15]:
def tokenize_text(text):
    words = word_tokenize(text)
    filtered_words = [word.lower() for word in words if word.isalpha()]
    return ' '.join(filtered_words) 

ro_df['text'] = ro_df['text'].apply(lambda x: tokenize_text(x))
md_df['text'] = md_df['text'].apply(lambda x: tokenize_text(x))


def TF(doc, term):
    words = doc.split()
    return words.count(term) / len(words)

In [98]:
ro_tf = np.zeros((len(ro_df), len(stopwords)))
for i in range(len(ro_df)):
    for j in range(len(stopwords)):
        ro_tf[i, j] = TF(ro_df['text'].iloc[i], stopwords[j])

ro_tf_average = np.sum(ro_tf, axis = 0)/ro_tf.shape[0]
sorted_indices = np.argsort(ro_tf_average)[::-1]

print("Cuvintele în ordinea mediei frecventei:")
for idx in sorted_indices:
    print(f"Cuvant: {stopwords[idx]}, Media TF: {ro_tf_average[idx]}")

Cuvintele în ordinea mediei frecventei:
Cuvant: de, Media TF: 0.04859260269997355
Cuvant: în, Media TF: 0.017633999465801926
Cuvant: la, Media TF: 0.014936163965123804
Cuvant: a, Media TF: 0.014184594473464616
Cuvant: să, Media TF: 0.013278481290340677
Cuvant: cu, Media TF: 0.013086409336629119
Cuvant: care, Media TF: 0.011267575137351165
Cuvant: o, Media TF: 0.011170366024112106
Cuvant: mai, Media TF: 0.010706444903439132
Cuvant: pe, Media TF: 0.010426836656860972
Cuvant: pentru, Media TF: 0.010157635379521787
Cuvant: din, Media TF: 0.009448286577606933
Cuvant: nu, Media TF: 0.008725943308117441
Cuvant: este, Media TF: 0.00812563731455136
Cuvant: un, Media TF: 0.007861773861536614
Cuvant: si, Media TF: 0.00760113470150522
Cuvant: se, Media TF: 0.007114430599315044
Cuvant: că, Media TF: 0.006997755156379331
Cuvant: in, Media TF: 0.005887279172558714
Cuvant: am, Media TF: 0.004994646538943568
Cuvant: ce, Media TF: 0.004885712377297183
Cuvant: ca, Media TF: 0.004505088193200761
Cuvant: s

In [99]:
md_tf = np.zeros((len(md_df), len(stopwords)))
for i in range(len(md_df)):
    for j in range(len(stopwords)):
        md_tf[i, j] = TF(md_df['text'].iloc[i], stopwords[j])

md_tf_average = np.sum(md_tf, axis = 0)/md_tf.shape[0]
sorted_indices = np.argsort(md_tf_average)[::-1]

print("Cuvintele în ordinea mediei frecventei:")
for idx in sorted_indices:
    print(f"Cuvant: {stopwords[idx]}, Media TF: {md_tf_average[idx]}")

Cuvintele în ordinea mediei frecventei:
Cuvant: de, Media TF: 0.0465451781204369
Cuvant: în, Media TF: 0.022223542295786755
Cuvant: a, Media TF: 0.017179519835010804
Cuvant: la, Media TF: 0.015803763339078052
Cuvant: să, Media TF: 0.01359802405548467
Cuvant: cu, Media TF: 0.013027475956163218
Cuvant: din, Media TF: 0.010575052545937049
Cuvant: mai, Media TF: 0.010088595167983496
Cuvant: o, Media TF: 0.00984487752726535
Cuvant: pentru, Media TF: 0.009674263754081865
Cuvant: care, Media TF: 0.009496984527770754
Cuvant: pe, Media TF: 0.009309052384190114
Cuvant: nu, Media TF: 0.007587829753275709
Cuvant: că, Media TF: 0.007228715523701089
Cuvant: un, Media TF: 0.00698570336532043
Cuvant: este, Media TF: 0.00576990477558932
Cuvant: se, Media TF: 0.0057050918225413395
Cuvant: am, Media TF: 0.004744196374038503
Cuvant: au, Media TF: 0.004204148963269366
Cuvant: fost, Media TF: 0.004092681507656019
Cuvant: şi, Media TF: 0.0037902047489960533
Cuvant: ce, Media TF: 0.0037609741079318976
Cuvant:

In [100]:
def IDF(term, corpus):
    nr_documents = len(corpus)
    doc_containing_term = sum(1 for doc in corpus if term in doc.split())
    doc_containing_term = doc_containing_term + 1
    return np.log(nr_documents/doc_containing_term)
    

In [102]:
ro_idf = np.zeros(len(stopwords))
for i in range(len(stopwords)):
    ro_idf[i] = IDF(stopwords[i],ro_df['text'])

ro_idf_sorted_indices = np.argsort(ro_idf)[::-1]

print("Cuvintele în ordinea IDF:")
for i in ro_idf_sorted_indices:
    print(f"Cuvant: {stopwords[i]},IDF:{ro_idf[i]}")

Cuvintele în ordinea IDF:
Cuvant: ţie,IDF:7.557994958530806
Cuvant: atit,IDF:7.557994958530806
Cuvant: atitea,IDF:7.557994958530806
Cuvant: atitia,IDF:7.557994958530806
Cuvant: voştri,IDF:7.557994958530806
Cuvant: aşadar,IDF:7.557994958530806
Cuvant: b,IDF:7.557994958530806
Cuvant: citiva,IDF:7.557994958530806
Cuvant: curînd,IDF:7.557994958530806
Cuvant: câţi,IDF:7.557994958530806
Cuvant: cîtva,IDF:7.557994958530806
Cuvant: cîţi,IDF:7.557994958530806
Cuvant: dintr-,IDF:7.557994958530806
Cuvant: geaba,IDF:7.557994958530806
Cuvant: graţie,IDF:7.557994958530806
Cuvant: sînteţi,IDF:7.557994958530806
Cuvant: sîntem,IDF:7.557994958530806
Cuvant: sînt,IDF:7.557994958530806
Cuvant: incit,IDF:7.557994958530806
Cuvant: sintem,IDF:7.557994958530806
Cuvant: puţina,IDF:7.557994958530806
Cuvant: printr-,IDF:7.557994958530806
Cuvant: oricînd,IDF:7.557994958530806
Cuvant: atita,IDF:7.557994958530806
Cuvant: câtva,IDF:7.557994958530806
Cuvant: ăştia,IDF:7.557994958530806
Cuvant: întrucît,IDF:7.55799495

In [104]:
md_idf = np.zeros(len(stopwords))
for i in range(len(stopwords)):
    md_idf[i] = IDF(stopwords[i],md_df['text'])

md_idf_sorted_indices = np.argsort(md_idf)[::-1]

print("Cuvintele în ordinea IDF:")
for i in md_idf_sorted_indices:
    print(f"Cuvant: {stopwords[i]},IDF:{md_idf[i]}")

Cuvintele în ordinea IDF:
Cuvant: inapoi,IDF:7.59337419312129
Cuvant: sînteţi,IDF:7.59337419312129
Cuvant: voştri,IDF:7.59337419312129
Cuvant: câtva,IDF:7.59337419312129
Cuvant: curînd,IDF:7.59337419312129
Cuvant: citiva,IDF:7.59337419312129
Cuvant: dintr,IDF:7.59337419312129
Cuvant: dintr-,IDF:7.59337419312129
Cuvant: ceilalti,IDF:7.59337419312129
Cuvant: h,IDF:7.59337419312129
Cuvant: intr,IDF:7.59337419312129
Cuvant: j,IDF:7.59337419312129
Cuvant: aşadar,IDF:7.59337419312129
Cuvant: atitia,IDF:7.59337419312129
Cuvant: atitea,IDF:7.59337419312129
Cuvant: atita,IDF:7.59337419312129
Cuvant: ălea,IDF:7.59337419312129
Cuvant: puţina,IDF:7.59337419312129
Cuvant: r,IDF:7.59337419312129
Cuvant: ului,IDF:7.59337419312129
Cuvant: ala,IDF:7.59337419312129
Cuvant: cîtva,IDF:7.59337419312129
Cuvant: nimeri,IDF:6.900227012561346
Cuvant: oricînd,IDF:6.900227012561346
Cuvant: oricît,IDF:6.900227012561346
Cuvant: printr-,IDF:6.900227012561346
Cuvant: geaba,IDF:6.900227012561346
Cuvant: sa-mi,IDF:6.9